<a href="https://colab.research.google.com/github/AndreaMorbello/Progetto-IA2/blob/main/Notebooks/Notebook-IA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook progetto baseline

## 1. Introduzione al progetto

### 1.1. Obiettivo del modello

L'obiettivo di questo progetto è quello di sviluppare un modello in grado di prevedere il rating ELO di un giocatore (in particolare del giocatore con i pezzi bianchi) a partire da feature estratte da una sua partita.
Questo modello, che chiameremo modello baseline, sarà poi confrontato con un secondo modello ... [DA COMPLETARE]


### 1.2. Presentazione del dataset

Il dataset che abbiamo utilizzato proviene da Kaggle:
https://www.kaggle.com/datasets/datasnaek/chess/code/data
Contiene 20058 partite di scacchi e contiene:
- id (Stringa alfanumerica identificativa della partita);
- rated (La partita è competitiva e comportarà una variazione dell'ELO del giocatore? True o False);
- created_at (Momento di inizio della partita in unix time);
- last_move_at (Momento di fine della partita in unix time);
- turns (Numero di turni di gioco intesi come coppie di mosse bianco-nero)
- victory_status (Come la partita si è conclusa: Scacco matto, stallo, tempo esaurito...);
- winner (Colore del giocatore che ha vinto la partita o draw (patta));
- increment_code (Minuti assegnati a ciascun giocatore all'inizio della partita  secondi assegnati dopo ciascuna mossa);
- white_id (Nome utente del giocatore bianco);
- white_rating (Punteggio ELO del giocatore bianco);
- black_id (Nome utente del giocatore nero);
- black_rating (Punteggio ELO del giocatore nero);
- moves (Tutte le mosse della partita in notazione scacchistica);
- opening_eco (codice identificativo dell'apertura della partita);
- opening_name (nome dell'apertura);
- opening_ply (singole mosse appartenenti alla fase di apertura della partita);

### 1.3. Importazione del Dataset




In [52]:
###INSERIRE CODICE PER IMPORTAZIONE

In [53]:
## Quando ultimeremo il codice sarebbe meglio non fare montare il drive.

import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

df=pd.read_csv("/content/drive/MyDrive/games.csv")
df_originale = df.copy()
df.info()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20058 entries, 0 to 20057
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              20058 non-null  object 
 1   rated           20058 non-null  bool   
 2   created_at      20058 non-null  float64
 3   last_move_at    20058 non-null  float64
 4   turns           20058 non-null  int64  
 5   victory_status  20058 non-null  object 
 6   winner          20058 non-null  object 
 7   increment_code  20058 non-null  object 
 8   white_id        20058 non-null  object 
 9   white_rating    20058 non-null  int64  
 10  black_id        20058 non-null  object 
 11  black_rating    20058 non-null  int64  
 12  moves           20058 non-null  object 
 13  opening_eco     20058 non-null  object 
 14  opening_name    20058 non-null  objec

## 2. Sviluppo del modello

### 2.1. Pulizia dei dati

a)  *Rimozione dei duplicati* (partite con il medesimo ID):
Notiamo che all'interno del modello erano presenti 945 partite con lo stesso ID.
Verifichiamo che siano effettivamente identiche e proseguiamo ad eliminare le righe ridondanti.

In [54]:
# Individuiamo gli ID che compaiono più di una volta
dup_ids = df['id'].value_counts()
dup_ids = dup_ids[dup_ids > 1]

print(f"Numero di ID duplicati: {len(dup_ids)}")

colonne_da_controllare = [c for c in df.columns if c != 'id']

identical_check = (
    df[df['id'].isin(dup_ids.index)]
    .groupby('id')[colonne_da_controllare]
    .nunique()
    .max(axis=1)
)

identical_check.value_counts()

Numero di ID duplicati: 813


,count
2,516
1,297


In [55]:
# Notiamo che alcune righe (516) non condividono valori identici per tutte le colonne, verifichiamo perché:
id_problematici = identical_check[identical_check > 1].index

df[df['id'].isin(id_problematici)] \
  .groupby('id')[colonne_da_controllare] \
  .nunique() \
  .head()

# Le colonne non identiche sono quelle di inizio e fine partita, dunque il problema
# è probabilmente dovuto a un problema di datatype o di misurazione nel dataset.
# Le partite sono semanticamente identiche in tutte le altre colonne.
# Possiamo quindi procedere con la rimozione dei duplicati.

,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
id,,,,,,,,,,,,,,,
079kHDqh,1,2,2,1,1,1,1,1,1,1,1,1,1,1,1
07e0uVvn,1,2,2,1,1,1,1,1,1,1,1,1,1,1,1
09fzI0ta,1,2,2,1,1,1,1,1,1,1,1,1,1,1,1
0CCos4zy,1,2,2,1,1,1,1,1,1,1,1,1,1,1,1
0LeqFhxc,1,2,2,1,1,1,1,1,1,1,1,1,1,1,1


In [56]:
df = df.drop_duplicates(subset='id').reset_index(drop=True)

b) *Eliminazione delle colonne del dataframe non necessarie*:
Saranno certamente ininfluenti gli ID dei giocatori, ID della partita e il nome dell'apertura (prediligeremo usare il più rigoroso opening ECO).
Il discorso è più complicato per quanto riguarda le mosse. Intuitivamente per valutare il livello di un giocatore queste sarebbero fondamentali, tuttavia, almeno per il modello baseline, proveremo a fare le nostre valutazioni senza prenderle in considerazione.

In [57]:
df.drop(columns=['white_id','black_id','id','opening_name','moves'])

,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_rating,black_rating,opening_eco,opening_ply
0,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,1500,1191,D10,5
1,True,1.504130e+12,1.504130e+12,16,resign,black,5+10,1322,1261,B00,4
2,True,1.504130e+12,1.504130e+12,61,mate,white,5+10,1496,1500,C20,3
3,True,1.504110e+12,1.504110e+12,61,mate,white,20+0,1439,1454,D02,3
4,True,1.504030e+12,1.504030e+12,95,mate,white,30+3,1523,1469,C41,5
...,...,...,...,...,...,...,...,...,...,...,...
19108,True,1.499791e+12,1.499791e+12,24,resign,white,10+10,1691,1220,A80,2
19109,True,1.499698e+12,1.499699e+12,82,mate,black,10+0,1233,1196,A41,2
19110,True,1.499698e+12,1.499698e+12,35,mate,white,10+0,1219,1286,D00,3
19111,True,1.499696e+12,1.499697e+12,109,resign,white,10+0,1360,1227,B07,4


c) *Trasformazione delle colonne di inizio e fine partita in una colonna di durata*

In [58]:
#Ora convertiamo le colonne di inizio e fine partita, che sono in formato Unix time, in formato datetime
#Dopodichè inseriremo una più semplice colonna di durata partita al posto delle due colonne iniziali

df_times = df[['created_at','last_move_at']].copy()

df_times['created_at_dt'] = pd.to_datetime(df_times['created_at']/1000, unit='s', origin='unix')

df_times['last_move_at_dt'] = pd.to_datetime(df_times['last_move_at']/1000, unit='s', origin='unix')

df_times['created_time'] = df_times['created_at_dt'].dt.time
df_times['last_move_time'] = df_times['last_move_at_dt'].dt.time
df_times['duration_seconds'] = (df_times['last_move_at_dt'] - df_times['created_at_dt']).dt.total_seconds()

#Adesso che abbiamo la durata della partita in secondi, inseriamo la colonna nel dataframe e rimuoviamo quelle iniziali

df=df.drop(columns=['created_at','last_move_at'])
df=df.join(df_times['duration_seconds'])

#Ora invece ci occupiamo della colonna 'increment_code', che è della forma "X+Y"
#dove X sono i minuti concessi a ciascun giocatore per giocare le proprie mosse e Y
#è il tempo che gli viene riaggiunto una volta giocata la mossa.
#Vogliamo separare questi due valori e tenerli in colonne separate

#Split di increment_code in due colonne
df[['base_time', 'increment_time']] = (
    df['increment_code'].str.split('+', expand=True).astype(int))

#Rimuoviamo la colonna originale
df = df.drop(columns=['increment_code'])

df.tail(50)

,id,rated,turns,victory_status,winner,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply,duration_seconds,base_time,increment_time
19063,2TPHqzpa,True,119,draw,draw,rb52,2049,denci,1895,d4 d5 c4 e6 Nc3 dxc4 e4 g6 Bxc4 Bg7 Nf3 c6 O-O...,D31,Queen's Gambit Declined: Queen's Knight Variation,5,1895.975,15,5
19064,d4Ujb4yY,True,118,resign,black,kam49,1798,rb52,2044,e4 d6 Nf3 Nf6 d3 c5 c4 g6 Nc3 Bg7 Be2 Nc6 a3 e...,B07,Pirc Defense #4,2,2231.065,15,5
19065,OK2C2s2g,True,58,resign,black,bhirghu,2001,rb52,2030,e4 d6 d4 Nf6 Bd3 g6 f4 e5 fxe5 dxe5 dxe5 Ng4 N...,B07,Pirc Defense,4,1819.828,15,5
19066,O4hEQMTb,True,60,resign,black,rqmik,1980,rb52,2017,f4 d5 Nf3 Nf6 e3 g6 b3 Bg7 Bb2 O-O Nc3 c5 Bd3 ...,A03,Bird Opening: Dutch Variation,2,629.199,5,5
19067,JwO79MMV,True,63,outoftime,white,rb52,2010,uburoijacques,1781,d4 Nf6 Nf3 g6 g3 Bg7 Bg2 d6 O-O O-O Nc3 c5 e4 ...,A49,Indian Game: Przepiorka Variation,5,813.539,5,5
19068,gkXZXc3K,True,87,outoftime,white,rb52,1998,joseigal,1920,d4 Nf6 Nf3 d5 c4 e6 Nc3 Bb4 Bg5 h6 Bxf6 Qxf6 e...,E20,Nimzo-Indian Defense: Ragozin Variation,8,782.868,5,5
19069,0FY0L9pp,True,50,resign,black,vahidddd,1684,rb52,1993,d4 d5 Nf3 Nf6 Bf4 Bf5 e3 c6 Nbd2 e6 Nh4 Be4 Bg...,D02,Queen's Pawn Game: London System,5,488.695,7,2
19070,VvoFXfsI,True,29,resign,white,rb52,1989,farshid_ghasemi,1653,d4 d5 c4 c6 Nc3 dxc4 a4 Bf5 e3 e6 Bxc4 Nf6 Nf3...,D19,Queen's Gambit Declined: Slav | Dutch Variation,15,315.656,7,2
19071,sQXtoPFG,True,46,mate,black,trakhtur2020,1838,rb52,1979,e4 d6 d4 Nf6 Nc3 g6 f4 Bg7 Nf3 c5 e5 Nfd7 exd6...,B09,Pirc Defense: Austrian Attack | Dragon Formation,10,409.982,7,2
19072,Tlb2deQW,True,41,mate,white,rb52,1972,stifi,1718,d4 d5 c4 c6 Nc3 e6 cxd5 cxd5 Nf3 f5 Bf4 Nf6 e3...,D31,Semi-Slav Defense: Accelerated Move Order,6,277.190,7,2


In [59]:
#Ora ci occupiamo di creare dummy per le colonne con valori categoriali: victory_status, winner e opening_eco
colonne_categoriali = ['winner', 'victory_status', 'opening_eco']

df = pd.get_dummies(df, columns=colonne_categoriali, dtype=int, drop_first=False)
df.tail()

,id,rated,turns,white_id,white_rating,black_id,black_rating,moves,opening_name,opening_ply,...,opening_eco_E81,opening_eco_E87,opening_eco_E88,opening_eco_E90,opening_eco_E91,opening_eco_E92,opening_eco_E94,opening_eco_E95,opening_eco_E97,opening_eco_E98
19108,EfqH7VVH,True,24,belcolt,1691,jamboger,1220,d4 f5 e3 e6 Nf3 Nf6 Nc3 b6 Be2 Bb7 O-O Be7 Ne5...,Dutch Defense,2,...,0,0,0,0,0,0,0,0,0,0
19109,WSJDhbPl,True,82,jamboger,1233,farrukhasomiddinov,1196,d4 d6 Bf4 e5 Bg3 Nf6 e3 exd4 exd4 d5 c3 Bd6 Bd...,Queen's Pawn,2,...,0,0,0,0,0,0,0,0,0,0
19110,yrAas0Kj,True,35,jamboger,1219,schaaksmurf3,1286,d4 d5 Bf4 Nc6 e3 Nf6 c3 e6 Nf3 Be7 Bd3 O-O Nbd...,Queen's Pawn Game: Mason Attack,3,...,0,0,0,0,0,0,0,0,0,0
19111,b0v4tRyF,True,109,marcodisogno,1360,jamboger,1227,e4 d6 d4 Nf6 e5 dxe5 dxe5 Qxd1+ Kxd1 Nd5 c4 Nb...,Pirc Defense,4,...,0,0,0,0,0,0,0,0,0,0
19112,N8G2JHGG,True,78,jamboger,1235,ffbob,1339,d4 d5 Bf4 Na6 e3 e6 c3 Nf6 Nf3 Bd7 Nbd2 b5 Bd3...,Queen's Pawn Game: Mason Attack,3,...,0,0,0,0,0,0,0,0,0,0


In [60]:
df.head()

,id,rated,turns,white_id,white_rating,black_id,black_rating,moves,opening_name,opening_ply,...,opening_eco_E81,opening_eco_E87,opening_eco_E88,opening_eco_E90,opening_eco_E91,opening_eco_E92,opening_eco_E94,opening_eco_E95,opening_eco_E97,opening_eco_E98
0,TZJHLljE,False,13,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,Slav Defense: Exchange Variation,5,...,0,0,0,0,0,0,0,0,0,0
1,l1NXvwaE,True,16,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,Nimzowitsch Defense: Kennedy Variation,4,...,0,0,0,0,0,0,0,0,0,0
2,mIICvQHh,True,61,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,King's Pawn Game: Leonardis Variation,3,...,0,0,0,0,0,0,0,0,0,0
3,kWKvrqYL,True,61,daniamurashov,1439,adivanov2009,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,Queen's Pawn Game: Zukertort Variation,3,...,0,0,0,0,0,0,0,0,0,0
4,9tXo1AUZ,True,95,nik221107,1523,adivanov2009,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,Philidor Defense,5,...,0,0,0,0,0,0,0,0,0,0
